<a href="https://colab.research.google.com/github/tomonari-masada/course2024-nlp/blob/main/finetuning_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 生成モデルをデータ分析に使う

### （実演） トピック抽出にLLMを使う
* 与えられたテキスト集合から、指定した個数のトピックを抽出する。
  * 各トピックは、そのトピックに属するテキストの集合として表される。
  * また、各トピックの内容を、20個の単語リストで表す。

### 準備

In [ ]:
!python -m spacy download ja_core_news_sm
#!pip install git+https://github.com/huggingface/transformers
!pip install -U transformers==4.40
!pip install -U bitsandbytes accelerate peft trl

### インポート

In [ ]:
import numpy as np
from tqdm.auto import tqdm

import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity

import torch
import torch.nn as nn
from datasets import load_dataset, concatenate_datasets
from transformers import (
    set_seed,
    BitsAndBytesConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
)
from transformers.modeling_outputs import ModelOutput
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

set_seed(123)

### データセット
* livedoorニュースコーパス
  * ９つのジャンルのニュース記事からなるテキストの集合。
  * ニュース記事のタイトル（短いテキスト）と内容（やや長いテキスト）からなる。

In [ ]:
dataset = load_dataset(
    "shunk031/livedoor-news-corpus",
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    random_state=42,
    shuffle=True,
    trust_remote_code=True,
)
num_categories = len(set(dataset["train"]["category"]))

max_seq_length = 512

In [ ]:
dataset["train"][0]

### モデルのロード
* `elyza/ELYZA-japanese-Llama-2-7b`というllama 2ベースの日本語対応LLMを使う。

In [ ]:
model_name = "elyza/ELYZA-japanese-Llama-2-7b"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_storage=torch.bfloat16,
)

pretrained = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_categories,
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_seq_length=max_seq_length)

tokenizer.pad_token = tokenizer.eos_token
pretrained.config.pad_token_id = pretrained.config.eos_token_id

* 分類の損失を自前で計算するため、新たにクラスを定義する。

In [ ]:
class LivedoorNet(nn.Module):
    def __init__(self, pretrained):
        super().__init__()
        self.pretrained = pretrained
        self.config = self.pretrained.config

    def forward(
        self,
        input_ids,
        category=None,
        attention_mask=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        inputs_embeds=None,
        labels=None,
    ):
        outputs = self.pretrained(
            input_ids,
            attention_mask=attention_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        loss_fct = nn.CrossEntropyLoss()
        loss = loss_fct(outputs.logits, category)
        return ModelOutput(
            loss=loss,
            logits=outputs.logits,
            past_key_values=outputs.past_key_values,
            hidden_states=outputs.hidden_states,
            attentions=outputs.attentions,
        )

model = LivedoorNet(pretrained)

### LoRAの設定

In [ ]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type="SEQ_CLS",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
        ],
)

* finetuningは実行せず、保存してあるLoRAを読み込むときは、以下のセルを実行する。

In [ ]:
model = PeftModel.from_pretrained(model, "models/lora/" + model_name)

In [ ]:
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    output_dir="outputs_cls",
    label_names=["category"],
    max_steps=500,
    eval_steps=100,
    logging_steps=100,
    save_steps=100,
    learning_rate=5e-5,
    evaluation_strategy="steps",
    logging_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    dataset_text_field="title",
    peft_config=peft_config,
)

* 以下の作業が必要。
  * Trainerのインスタンスを作るとtokenizationに無関係なfieldは削除されてしまう。

In [ ]:
trainer.train_dataset = trainer.train_dataset.add_column("category", dataset["train"]["category"])
trainer.eval_dataset = trainer.eval_dataset.add_column("category", dataset["validation"]["category"])

* 評価用のヘルパ関数

In [ ]:
def accuracy(trainer, dataset, batch_size=4):
    trainer.model.eval()
    num_correct_answers = 0
    num_answers = 0
    for i in tqdm(range(0, len(dataset), batch_size)):
        examples = dataset[i:i+batch_size]
        encodings = trainer.tokenizer(
            examples["title"],
            padding=True,
            return_tensors="pt",
            )
        category = torch.tensor(examples["category"])
        with torch.no_grad():
            outputs = trainer.model(**encodings, category=category)
        num_correct_answers += (outputs.logits.argmax(-1) == category).sum()
        num_answers += len(examples["category"])
    trainer.model.train()
    return num_correct_answers / num_answers

* finetuning前に分類性能を正解率で評価

In [ ]:
accuracy(trainer, dataset["validation"])

* fine-tuningの実行
  * この実行例で必要なGPUのメモリは10GB未満。

In [ ]:
#trainer.train()

In [ ]:
#accuracy(trainer, dataset["validation"])

In [ ]:
#trainer.model.save_pretrained("models/lora/" + model_name)

### テキストの埋め込みを求めるヘルパ関数

In [ ]:
def embed(trainer, dataset, batch_size=4):
    trainer.model.eval()
    categories = []
    pooled_hidden_states = []
    for i in tqdm(range(0, len(dataset), batch_size)):
        examples = dataset[i:i+batch_size]
        encodings = trainer.tokenizer(
            examples["title"],
            padding=True,
            return_tensors="pt",
            )
        categories += list(examples["category"])
        with torch.no_grad():
            outputs = trainer.model.pretrained.model(**encodings)
        pad_token_id = trainer.model.pretrained.config.pad_token_id
        input_ids = encodings.input_ids
        sequence_lengths = torch.eq(input_ids, pad_token_id).int().argmax(-1) - 1
        sequence_lengths = sequence_lengths % input_ids.shape[-1]
        temp_batch_size = input_ids.shape[0]
        pooled_hidden_state = outputs.last_hidden_state[
            torch.arange(temp_batch_size, device=outputs.last_hidden_state.device),
            sequence_lengths]
        pooled_hidden_states.append(pooled_hidden_state.float().cpu().numpy())
    trainer.model.train()
    return categories, np.concatenate(pooled_hidden_states)

### 全テキストの埋め込み

In [ ]:
full_dataset = concatenate_datasets([dataset["train"], dataset["validation"], dataset["test"]])
categories, embeddings = embed(trainer, full_dataset)

In [ ]:
embeddings.shape

### クラスタのラベルとして使う語彙の作成
* spaCyで記事の全タイトルを形態素解析する。
  * 名詞、動詞、固有名詞だけを残す。動詞は原形に直す。

In [ ]:
label_pos_tags = ["NOUN", "VERB", "PROPN"]

nlp = spacy.load("ja_core_news_sm")
corpus = []
for text in tqdm(full_dataset["title"]):
    corpus.append(" ".join([token.lemma_ for token in nlp(text) if token.pos_ in label_pos_tags]))

### TF-IDFの計算
  * `TfidfVectorizer`の`min_df`パラメータは適当に調節する。
  * クラスタのラベリングに向かないマイナーな単語が含まれないようにする。

In [ ]:
vectorizer = TfidfVectorizer(min_df=10, lowercase=False)
X = vectorizer.fit_transform(corpus).toarray()
vocab = np.array(vectorizer.get_feature_names_out())

### クラスタのラベルとして使う語彙の埋め込み

In [ ]:
vocab_embeddings = np.dot((X / X.sum(0)).T, embeddings)

### テキストのクラスタリング

* クラスタ数は適当に決める。

In [ ]:
n_clusters = 20
kmeans = KMeans(n_clusters=n_clusters, n_init='auto', random_state=123)
kmeans.fit(embeddings)
centers = kmeans.cluster_centers_

* クラスタのサイズの分布を調べる。

In [ ]:
unique, counts = np.unique(kmeans.labels_, return_counts=True)
size_dict = dict(zip(unique, counts))
print(size_dict)

### 各クラスタの重心に近い順に20個の単語を列挙
* これが各クラスタのラベルになる。

In [ ]:
similarities = cosine_similarity(vocab_embeddings, centers)

for i in range(similarities.shape[-1]):
    indices = np.argsort(- similarities[:,i])
    print(" ".join(list(vocab[indices[:20]])))
    print("-"*100)